In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir('/kaggle/input/asl-signs'))
   

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1. Necessary Imports 

In [ ]:
# Kaggle already includes these; no need to install
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")


2. Global configuiration 

In [ ]:
BASE_DIR = "/kaggle/input/asl-signs"  # or your local path
ROWS_PER_FRAME = 543
NUM_CLASSES = 250
MAX_LEN = 384  # Frames to keep per sample
PAD = -100.0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


3. Landmark Selection for Feature Extraction

In [ ]:
# Define landmark indices from MediaPipe
LIP = [61, 185, 40, 39, 37, 267, 269, 270, 409, 291, 146, 91, 181, 84, 17, 314, 405, 321, 375]
LHAND = np.arange(468, 489).tolist()
RHAND = np.arange(522, 543).tolist()
NOSE = [1, 2, 98, 327]
REYE = [33, 7, 163, 144, 145]
LEYE = [263, 249, 390, 373, 374]

POINT_LANDMARKS = LIP + LHAND + RHAND + NOSE + REYE + LEYE
NUM_NODES = len(POINT_LANDMARKS)
CHANNELS = 6 * NUM_NODES


4. Load and Encode Labels 

In [ ]:
df = pd.read_csv(f"{BASE_DIR}/train.csv")
le = LabelEncoder()
df['label'] = le.fit_transform(df['sign'])
sign_to_index = {s: i for i, s in enumerate(le.classes_)}


5. Custom Functions for Nan-Safe Mean and Std

In [ ]:
# Compute mean across dimension while ignoring NaN values
def torch_nan_mean(x, dim=0, keepdim=False):
    mask = ~torch.isnan(x)
    x = torch.where(mask, x, torch.zeros_like(x))
    count = mask.sum(dim=dim, keepdim=keepdim).clamp(min=1)
    return x.sum(dim=dim, keepdim=keepdim) / count

# Compute standard deviation across dimension while ignoring NaNs
def torch_nan_std(x, center=None, dim=0, keepdim=False):
    if center is None:
        center = torch_nan_mean(x, dim=dim, keepdim=True)
    d = x - center
    variance = torch_nan_mean(d * d, dim=dim, keepdim=keepdim)
    return torch.sqrt(variance)


6. Preprocessing Module (Normalization + Feature Engineering)

In [ ]:
class Preprocess(nn.Module):
    def __init__(self, max_len=None, point_landmarks=None):
        super().__init__()
        self.max_len = max_len
        self.point_landmarks = point_landmarks

    def forward(self, inputs):
         # Ensure batch dimension exists
        if inputs.dim() == 3:
            x = inputs.unsqueeze(0)
        else:
            x = inputs

        mean = torch_nan_mean(x[:, :, [17], :], dim=(1, 2), keepdim=True)

        mean = torch.where(torch.isnan(mean), torch.tensor(0.5, device=x.device), mean)

        x = x[:, :, self.point_landmarks, :]
        std = torch_nan_std(x, mean, dim=(1, 2), keepdim=True)

        x = (x - mean) / std

        if self.max_len:
            x = x[:, :self.max_len]
        length = x.shape[1]

        x = x[..., :2]  # drop z
         # Compute velocity (1st derivative) and acceleration (2nd derivative)
        dx = torch.cat([x[:, 1:] - x[:, :-1], torch.zeros_like(x[:, :1])], dim=1)
        dx2 = torch.cat([x[:, 2:] - x[:, :-2], torch.zeros_like(x[:, :2])], dim=1)
         # Concatenate position, velocity, and acceleration as features
        x = torch.cat([
            x.reshape(-1, length, 2 * len(self.point_landmarks)),
            dx.reshape(-1, length, 2 * len(self.point_landmarks)),
            dx2.reshape(-1, length, 2 * len(self.point_landmarks)),
        ], dim=-1)

        return torch.nan_to_num(x, nan=0.0)


7.  Wrapper Function for Preprocessing + Label Conversion

In [ ]:
def preprocess(x, sign, max_len=MAX_LEN):
    x = torch.tensor(x, dtype=torch.float32)
    x = x.view(-1, ROWS_PER_FRAME, 3)  # Convert to (frames, landmarks, xyz)

    if x.shape[0] < max_len:
        pad_len = max_len - x.shape[0]
        pad = torch.zeros((pad_len, ROWS_PER_FRAME, 3))
        x = torch.cat([x, pad], dim=0)
    else:
        x = x[:max_len]
    # Apply preprocessing (normalization, landmark selection, velocity features)
    processor = Preprocess(max_len=max_len, point_landmarks=POINT_LANDMARKS)
    x = processor(x.unsqueeze(0)).squeeze(0)

    label = sign_to_index[sign]
    return x, label


8. PyTorch Dataset Class

In [ ]:
class SignLanguageDataset(Dataset):
    def __init__(self, df, max_len=384,augment=False, transform=None):
        self.df = df
        self.max_len = max_len
        self.augment = augment
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(BASE_DIR, row['path'])
        data = pd.read_parquet(path)
        coords = data[['x', 'y', 'z']].values
        # Apply transformation (preprocessing)
        x, y = self.transform(coords, row['sign'], max_len=self.max_len)
        return x.float(), torch.tensor(y).long()


9. DataLoader Utility Function

In [ ]:
def get_pytorch_dataset(df, batch_size=64, max_len=384,augment=False, transform=None):
    # Instantiate the dataset class with preprocessing logic
    dataset = SignLanguageDataset(df, max_len=max_len, transform=transform)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2,pin_memory=True)


10. Train-Validation Split + DataLoader Creation

In [ ]:
# Stratified split of the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, stratify=df['sign'], random_state=42)

train_loader = get_pytorch_dataset(train_df, batch_size=64, max_len=384, transform=preprocess)
val_loader = get_pytorch_dataset(val_df, batch_size=64, max_len=384, transform=preprocess)


11. CNN + LSTM Model Definition

In [ ]:
import torch.nn as nn

class CNNLSTMModel(nn.Module):
    def __init__(self, input_dim=708, num_classes=250, hidden_size=256):
        super(CNNLSTMModel, self).__init__()
         # 1D Convolution layer over time dimension (sequence modeling)
        self.conv1 = nn.Conv1d(input_dim, 256, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(0.3)
         # Bidirectional LSTM for capturing temporal dependencies
        self.lstm = nn.LSTM(input_size=256, hidden_size=hidden_size, batch_first=True, bidirectional=True)
        # Fully connected layers for classification
        self.fc1 = nn.Linear(hidden_size * 2, 256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)
        self.fc_out = nn.Linear(256, num_classes)

    def forward(self, x):  # x: (batch, seq_len, features)
        x = x.transpose(1, 2)  # → (batch, features, seq_len) for Conv1D
        x = self.dropout1(self.bn1(self.relu1(self.conv1(x))))
        x = x.transpose(1, 2)  # → (batch, seq_len, features) for LSTM

        lstm_out, _ = self.lstm(x)  # → (batch, seq_len, hidden*2)
        x = lstm_out[:, -1, :]  # Use last timestep

        x = self.dropout2(self.relu2(self.fc1(x)))
        return self.fc_out(x)      #Final prediction layer


12. Model Evaluation 

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(y_true, y_pred):
    y_true = y_true.cpu().numpy()
    y_pred = y_pred.cpu().numpy()

    preds = y_pred.argmax(axis=1)
    acc = accuracy_score(y_true, preds)
    f1 = f1_score(y_true, preds, average='macro')

    return {
        "accuracy": acc,
        "f1_score": f1,
        "score": (acc + f1) / 2  # Final metric
    }


In [ ]:
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm

def train_model(model, dataloader, validloader, optimizer, criterion, epochs=10):
    best_score = 0
    scaler = GradScaler()  # For mixed precision

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        print(f"\nEpoch {epoch + 1}/{epochs}")
        for inputs, labels in tqdm(dataloader, desc="Training", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with autocast():  # Mixed precision context
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()

        print(f"Train Loss: {total_loss / len(dataloader):.4f}")

        # 🔍 Validation (no need for AMP here)
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in validloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                all_preds.append(outputs.cpu())
                all_labels.append(labels.cpu())

        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)

        metrics = compute_metrics(all_labels, all_preds)
        print(f"Validation Accuracy: {metrics['accuracy']:.4f}, F1 Score: {metrics['f1_score']:.4f}, Combined Score: {metrics['score']:.4f}")

        if metrics['score'] > best_score:
            best_score = metrics['score']
            torch.save(model.state_dict(), "best_model.pt")
            print("✅ New best model saved!")


In [ ]:
# Load dataset
train_df, val_df = train_test_split(df, test_size=0.1, stratify=df['sign'], random_state=42)

train_loader = get_pytorch_dataset(train_df, batch_size=64, max_len=384, augment=True, transform=preprocess)
val_loader = get_pytorch_dataset(val_df, batch_size=64, max_len=384, augment=False, transform=preprocess)

# Initialize model
model = CNNLSTMModel(input_dim=450, num_classes=250).to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train
train_model(model, train_loader, val_loader, optimizer, criterion, epochs=3)
